# Reading SPH data

Here we demonstrate reading in a collection of Phantom HDF dump files, on which we will perform visualization and analysis.

In [ ]:
from plonk.dump import Dump

Read the dump files into a list. Each element of the list is a Plonk Dump object.

In [ ]:
dump_file_names = ['data/disc_00000.h5']

dumps = list()

for dump_file_name in dump_file_names:
    print(f'Reading {dump_file_name}...')
    dump = Dump(dump_file_name)
    dumps.append(dump)

We can access the dump parameters, i.e. the Phantom header. This is a dictionary with keys given by the Phantom header. For example we can access `hfact`, `alpha`, etc.

In [ ]:
hfact = dumps[0].parameters["hfact"]
alpha = dumps[0].parameters["alpha"]

print(f'hfact = {hfact:.1f}')
print(f'alpha = {alpha:.3f}')

There is also a dictionary for units where the value is the value of the code units in cgs units.

In [ ]:
for key in dumps[0].units:
    print(f'{key:20} {dumps[0].units[key]}')

The particle data is stored as a Pandas dataframe. Every particle has an `itype`, positions (as `x`, `y`, `z`), mass, smoothing length, and density (computed from the smoothing length), as well as any other quantities stored in the dump file, such as velocity (as `vx`, `vy`, `vz`), dust fraction, and so on. For example, the following shows the first 5 particles of the first dump file.

In [ ]:
dumps[0].particles.iloc[:5]

Similarly we have access to the sink particle data stored as a Pandas dataframe.

In [ ]:
dumps[0].sinks

# Analyzing SPH data

We can perform analysis on the SPH data. The function `disc_analysis` is equivalent to the Phantom analysis module available in `analysis_disc.f90`.

In [ ]:
from plonk.analysis.disc import disc_analysis

This analysis assumes a single disc around a single star (represented as a sink particle). We need to define the number of radial bins to average our data, as well as the inner and outer disc radius.

In [ ]:
number_radial_bins = 200
radius_in          = 1
radius_out         = 150

The analysis produces a list of Pandas dataframes `radial_averages`. Each datafram has index associated with the radial bin.

In [ ]:
radial_averages = list()
particles       = list()
sinks           = list()

for dump in dumps:

    print('\nPerforming disc analysis...\n')
    radial_averages_ = disc_analysis( radius_in          = radius_in,
                                      radius_out         = radius_out,
                                      number_radial_bins = number_radial_bins,
                                      dump               = dump )

    radial_averages.append(radial_averages_)

Now we can use the data in radial_averages to plot radially averaged quantities such as the surface density profile, or the disc aspect ratio, for example.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)

for df in radial_averages:
    ax[0].plot(df['R'], df['sigma'])
ax[0].set_xlabel('radius')
ax[0].set_ylabel('surface density')

for df in radial_averages:
    ax[1].plot(df['R'], df['H'])
ax[1].set_xlabel('radius')
ax[1].set_ylabel('scale height')

# Visualizing SPH data

We can use highly-optimized Splash interpolation routines to visualize SPH data.

In [ ]:
from plonk.visualization.image import Image

We create a list of Image objects. One for each dump file.

In [ ]:
images = list()
for dump in dumps:
    images.append(Image(dump))

Then we plot, for example, the columns density:

In [ ]:
images[0].plot(render='rho', render_fraction_max=0.05)

We can overlay vector fields:

In [ ]:
images[0].plot(render='rho', vector='v', render_fraction_max=0.05)

We can rotate about an arbitrary axis:

In [ ]:
images[0].plot(render='rho', rotation_angle=np.pi/3, rotation_axis=[1, 1, 0],
               render_fraction_max=0.05)

We can take a cross-section:

In [ ]:
images[0].plot(render='rho', rotation_angle=np.pi/2, rotation_axis=[1, 0, 0],
               cross_section=True, render_fraction_max=0.01)

We can plot the particles. (This is somewhat slow.)

In [ ]:
images[0].plot(horizontal_range=[50, 150], vertical_range=[50, 150])